# Interactive tutorial: best practices for supervised machine learning in microbiomics

This Colab notebook is an accompaniment to the paper "Best practices for supervised machine learning in microbiomics" (**ADD CITATION**).

In this tutorial, we will develop a machine learning (ML) classifier that can predict whether an individual has schizophrenia based on the composition of their fecal microbiota. Data was sourced from [Zhu, Feng, et al. "Metagenome-wide association of gut microbiome features for schizophrenia." Nature communications 11.1 (2020): 1612.](https://www.nature.com/articles/s41467-020-15457-9) For more information about the clinical trial under which data was collected, including the study design and eligibility criteria, see [clinicaltrials.gov, identifier NCT02708316](https://clinicaltrials.gov/ct2/show/NCT02708316?term=NCT02708316&draw=2&rank=1).

If you're new to Colab:

- To run this notebook, you'll need to make your own copy by going to File > Save a copy in Drive.
- Code in cells can be run by clicking the play button on the lefthand side of the cell with your cursor or by using the keyboard shortcut "Command/Ctrl+Enter".
- To learn more about Colab, check out the [documentation](https://colab.research.google.com/#scrollTo=-Rh3-Vt9Nev9) and [FAQ](https://research.google.com/colaboratory/faq.html).

In [ ]:
# automatically upload modules
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import defaultdict
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from ml4microbiome import data_visualization, pre_process, train

In [ ]:
BASE_DIR = "./"  # change to your path

DATA_PATH = "zhu_metaphlan_read_counts.txt"
METADATA_PATH = "zhu_supplemental.tsv"

## Experimental design

**Goals:** 
1. Build an ML classifier that can distinguish between individuals with vs without schizophrenia using microbiome and/or clinical data. 
2. Understand why/how the model works.
3. Investigate which population segments it performs more or less well on. 

The data will be pre-processed as follows. While some may be more or less important for different projects and methodologies (e.g.: feature selection), we perform them here to illustrate how they can be incorporated into experimental design without data leakage. 
- CLR transform microbiome data
- Imputation (**add type**) of missing metadata entries
- Feature scaling (**add type**) on numerical metadata features
- Feature selection (**add type**)

We will compare the performance of models trained using the following parameters:
- Different types of features (metadata only, microbiome only, combined)
- Different ML algorithms (random forest, L1 logistic regression, LightGBM)
- Different taxonomic levels (species, genus, family, all taxonomic levels)

These comparisons will be performed using repeated, nested cross-validation (CV). Importantly, model selection will be performed on a nested CV set so that the performance of the final model will be evaluated on previously unseen data. 

Once our best model is selected, we will endeavour to explain how/why it makes predictions using Shapley Additive Explanations (SHAP). 

Finally, we will perform an error analysis to better understand under what circumstances our model may perform more or less well. 

## Data pre-processing

Raw reads were downloaded from the [European Nucleotide Archive Project PRJEB29127](https://www.ebi.ac.uk/ena/browser/view/PRJEB29127). Quality control analysis with FastQC v0.11.9 (Andrews, 2010) and MultiQC v1.12 (Ewels *et al*., 2016), revealing that reads in the dataset had high quality scores across their lengths and that there was minimal adapter content remaining. As such, we proceeded to remove human contamination by mapping reads against human reference genome [GRCh38.p14](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCF_000001405.40/) using bowtie2 vX (Langmead & Salzberg, 2012). 

The taxonomic composition of samples was inferred  using [Metaphlan4](https://huttenhower.sph.harvard.edu/metaphlan/) (Segata *et al*., 2012; Blanco-Miguez *et al*., 2022) and the Jan 2021 ChocoPhlAn3 database as a reference (citation). Metaphlan4 was run with the -t rel_ab_w_read_stats flag to obtain read counts per taxon - this will later be normalized using the CLR transformation. One of the strengths of shotgun sequencing is that it can be used to profile the bacteria, archaea, eukaryotes, and viruses in a sample, all of which can influence community ecology and host phenotype. By default, Metaphlan4 will profile bacterial, archaeal, and eukaryotic taxa. We added the --add_viruses flag to include information about viral taxa present in metagenomes. 

**Best practices:**

- For software tools, databases, and reference genomes, note the version used. In rare instances where the distributor does not specify explicit versions, state the date on which you accessed them.
- For software tools, describe whether you used default parameters vs customized settings. If the latter, explain which settings you used and why.

In [ ]:
df = pd.read_csv(BASE_DIR + DATA_PATH, sep="\t", header=1, index_col=0)
df.head(10)

It is essential that ML practitioners understand the format of raw input data before developing models. Here, we are working with the profile file output by Metaphlan4. 

Each column represents a sample and each row represents a taxon that was identified in the dataset. Values in cells represent the number of reads per sample attributed to a given taxon. For every taxonomic level (kingdom, phylum, ... genus, species), metaphlan outputs the read counts of all taxa identified at that level. Taxonomic levels are denoted by "k__" for kingdom, "p__" for phylum, ..., "g__" for genus, "s__" for species. This means that some rows will be redundant. For example, reads asigned to the species *Escherichia coli* will also be counted under genus *Escherichia*, family Enterobacteriaceae, Order Enterobacterales, Phylum Pseudomonadota, and Kingdom Bacteria. 

For more information, see the [Metaphlan4 tutorial](https://github.com/biobakery/biobakery/wiki/metaphlan4#131-the-metaphlan-taxonomic-profile).

In [ ]:
df_metadata = pd.read_csv(
    BASE_DIR + METADATA_PATH, skiprows=3, index_col=0, sep="\t"
).iloc[:, :-4]
df_metadata = df_metadata.replace(
    to_replace="Xi'An Mental Health Center", value="Xi'an"
)
df_metadata.head()

In [ ]:
df_metadata.isnull().any().any()

Looking at the metadata, a couple things to note are:
- Some metadata columns contain NaNs, we will perform imputation to deal with those later. Importantly, imputation must be done after the training-test split for model evaluation, so as to avoid data leakage.
- Categorical features such as "Gender" should be one hot encoded.
- Behavioural features may be affected by whether or not an individual is aware that they have diagnosed schizophrenia. For example, an individual with severe schizophrenia may be more or less likely to live in a certain type of housing ("Dwelling condition"). Since we want to develop a classifier that would theoretically be able to predict whether individuals without diagnosed schizophrenia may have schizophrenia, we will not include behavioural features in our ML model.

**Best practices:**  
- Understand the format of data and metadata that will be used as input to the ML experiment
- Carefully consider whether metadata could include features that can result in data leakage

## Data Exploration

Before designing our ML experiment, we will perform data exploration to understand the dataset available. 

This includes characterizing:
- Dataset size
- Demographics of the population sampled
- Technical features relating to how the data was generated (e.g.: collection, sampling, and processing methods)
- Properties of the sequenced dataset, such as the # of species identified by Metaphlan4

In [ ]:
# Note: let's not do any data exploration until we see what the ML results look like
# and if we want to use this dataset for the final tutorial

In [ ]:
fig = data_visualization.pie_plot_hospital(df_metadata)

In [ ]:
# TODO: data exploration and visualization
# E.g.: participant demographics (age, sex)
# E.g.: distribution of patients (and diagnoses) per hospital center

## Pre-process data 

To account for the compositionality of microbiomics data, we will apply the centered log ratio (CLR) transformation. We will not use relative abundance because ____. To avoid data leakage, other types of data pre-processing (feature scaling, feature selection) must be applied only once the training-test split has been made for evaluating model performance. The difference between CLR vs something like feature scaling is that CLR is calculate per sample, whereas the statistics used for feature scaling are calculated across multiple samples. 

In [ ]:
tax_levels = ["all", "species", "genus", "family"]

In [ ]:
data_dict = defaultdict(lambda: defaultdict())

In [ ]:
# Create dfs with specific taxonomic levels (e.g.: species only)
# CLR transform microbiome datasets
for tax_level in tax_levels[1:]:
    _ = pre_process.get_tax_level(df, tax_level)
    data_dict["microbiome_only"][tax_level] = pre_process.clr_transform(_).T

data_dict["microbiome_only"]["all"] = pre_process.clr_transform(df).T

In [ ]:
# Format target labels
# Target = whether or not a study participant has schizophrenia
y = df_metadata["Group"].replace(to_replace="HC", value="NS")  # NS = No schizophrenia

In [ ]:
# For metadata, we will retain only features that derive from objective clinical measurements
X_metadata_only = df_metadata[
    [
        "Age",
        "BMI",
        "Height(cm)",
        "Weight(kg)",
        "Pulse(c. p. m)",
        "Breathe(c. p. m)",
        "Systolic pressure(mmHg)",
        "Diastolic pressure(mmHg)",
        "Tryptophane(μM)",
        "Glutamic acid(μM)",
        "Tyrosine(μM)",
        "Phenylalanine(μM)",
        "Dopamine(ng/ml)",
        "Gamma-aminobutyric acid  (GABA)(ng/L)",
        "Serotonin(ng/ml)",
        "Kynurenine (KYN)(nmol/L)",
        "Kynurenic acid (KYNA)(nmol/L)",
    ]
]
metadata_continuous_cols = X_metadata_only.columns

In [ ]:
# Now we'll one hot encode "Gender" -- a categorical feature
# rows are samples, columns are features
gender = pd.get_dummies(df_metadata["Gender (1:male, 2:female)"])
X_metadata_only = pd.merge(X_metadata_only, gender, left_index=True, right_index=True)
data_dict["metadata_only"]["all"] = X_metadata_only

#### TODO: add headers back in for columns

In [ ]:
# # Merge metadata with microbiome data (at different taxonomic levels)
data_dict = pre_process.merge_metadata_microbiome(data_dict, tax_levels)

## Perform feature selection, model training, and hyperparameter tuning using nested 10-fold cross-validation

[Add text explaining strengths of our experimental design]

In [ ]:
# TODO: Section on choosing k for k-fold CV
# Talk about why you might want k=5 vs k=10, some combination thereof, etc
# Think through how many samples you have, and how many of each class
# Here, 171 samples - 81 HC, 90 SCZ (df_metadata["Group"].value_counts())
# Stratified 10-fold CV:
#   17*9=153 for training fold, 17 samples for test fold
# When nested, with 5-fold CV:
#   153/5 = 30 ---> 30*4=120 for training, 30 for test -- ~15 HC and 15 SCH
# For a smaller dataset, however, 10-fold CV might not make sense,
#   especially if there is a strong class imbalance

In [ ]:
# Record time and memory usage to record models?

In [ ]:
# Encode labels
encoder = LabelEncoder()
y_encoded = pd.Series(encoder.fit_transform(y))

In [ ]:
data_types = ["metadata_only", "microbiome_only", "metadata_microbiome"]
learning_algs = ["random_forest", "lightGBM", "logistic_regression_L1"]
reps = 10
test_size = 0.2
params_distributions = {
    "random_forest": {
        "n_estimators": np.arange(50, 550, 50),
        "max_features": ["log2", "sqrt", None],
        "min_samples_split": np.arange(2, 10, 2),
        "min_samples_leaf": np.arange(2, 12, 2),
        "bootstrap": [True, False],
        "max_depth": np.arange(5, 11),
    },
    "lightGBM": {
        "learning_rate": np.linspace(0.01, 0.5, 50),
        "objective": ["binary"],
        "boosting_type": ["gbdt"],
        "num_leaves": np.arange(25, 200, 25),
        "max_depth": np.arange(1, 10),
        "subsample": np.arange(0.5, 1.0, 0.1),
        "subsample_freq": np.arange(2, 10, 2),
        "colsample_bytree": np.arange(0.5, 1.0, 0.1),
        "min_child_weight": np.arange(25, 100, 25),
        "reg_alpha": np.arange(25, 100, 25),
        "max_bin": np.arange(155, 355, 50),
        "min_child_samples": np.arange(10, 50, 10),
    },
    "logistic_regression_L1": {
        "max_iter": [10000],
        "penalty": ["l1"],
        "C": np.linspace(0.1, 100, 50),
        "solver": ["liblinear", "saga"],
    },
}

outer_results = defaultdict(
    lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
)
inner_results = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    )
)

for data_type in data_types:
    for tax_level in tax_levels:
        for alg in learning_algs:
            # For metadata_only, there are no taxonomic levels so only train once
            if data_type == "metadata_only" and tax_level != "all":
                continue

            print(f"Data type: {data_type}")
            print(f"Taxonomic level: {tax_level}")
            print(f"Algorithm: {alg}")

            for outer_iter_no in range(reps):
                # Define outer loop training and test sets
                (
                    outer_X_train,
                    outer_X_test,
                    outer_y_train,
                    outer_y_test,
                ) = train_test_split(
                    data_dict[data_type][tax_level],
                    y_encoded,
                    test_size=test_size,
                    stratify=y_encoded,
                    random_state=outer_iter_no,
                )

                # Median imputation based on outer cv set
                # To avoid data leakage:
                #     Calculate median values on the TRAINING set
                #     Use TRAINING set medians to impute TRAINING and TEST set missing values
                outer_X_train = outer_X_train.fillna(outer_X_train.median())
                outer_X_test = outer_X_test.fillna(outer_X_train.median())

                # Feature scaling based on outer cv set
                # (Probably not actually that helpful here, but good for demonstration purposes)
                # To avoid data leakage:
                #    Calculate scaling stats on the TRAINING set alone
                #    Use TRAINING set stats to scale TRAINING and TEST set values
                if data_type != "microbiome_only":
                    outer_X_train, outer_X_test = train.scale_features(
                        outer_X_train, outer_X_test, metadata_continuous_cols
                    )

                # To do: add feature selection step

                # Store AUROCs for inner test set
                # And the best hyper-parameters
                for inner_iter_no in range(reps):
                    # Define inner loop training and test sets
                    (
                        inner_X_train,
                        inner_X_test,
                        inner_y_train,
                        inner_y_test,
                    ) = train_test_split(
                        outer_X_train,
                        outer_y_train,
                        test_size=test_size,
                        stratify=outer_y_train,
                        random_state=inner_iter_no,
                    )

                    best_model, best_params = train.tune_model(
                        alg,
                        params_distributions[alg],
                        "roc_auc",
                        inner_X_train,
                        inner_y_train,
                        inner_iter_no,
                    )
                    auc = train.test_model(best_model, inner_X_test, inner_y_test)
                    inner_results[data_type][tax_level][alg][outer_iter_no][
                        "AUROC"
                    ].append(auc)
                    inner_results[data_type][tax_level][alg][outer_iter_no][
                        "best_params"
                    ].append(best_params)

                # Back to outer training-test split
                # For a given data_type, tax_level, and alg, identify the best model stored in inner_results
                aurocs_np = np.array(
                    inner_results[data_type][tax_level][alg][outer_iter_no]["AUROC"]
                )
                median_auroc = np.median(aurocs_np)
                max_auroc_index = aurocs_np.argmax()
                params = inner_results[data_type][tax_level][alg][outer_iter_no][
                    "best_params"
                ][max_auroc_index]

                outer_results[data_type][tax_level][alg][
                    "inner_median_valid_AUROC"
                ].append(median_auroc)
                outer_results[data_type][tax_level][alg]["params"].append(params)

                # Train a model with the best parameters
                model = train.train_model(
                    alg, outer_X_train, outer_y_train, outer_iter_no, params
                )

                # Evaluate and record its performance on the outer test set in outer_results
                auc = train.test_model(model, outer_X_test, outer_y_test)
                outer_results[data_type][tax_level][alg]["AUROC"].append(auc)

## Select best model based on CV results

In [ ]:
# For each trained model, calculate median CV result plus standard deviation
# Use median AUROC to select best model

# Find maximum median AUROC
max_median_auroc = 0

for data_type_key, data_type_value in outer_results.items():
    for tax_level_key, tax_level_value in data_type_value.items():
        for alg_key, alg_value in tax_level_value.items():
            max_auroc = max(alg_value["inner_median_valid_AUROC"])
            if max_auroc > max_median_auroc:
                max_median_auroc = max_auroc

max_median_auroc

In [ ]:
# Find all models that have max_median_auroc

best_models_info = []
tolerance = 1e-6  # 0.000001

for data_type_key, data_type_value in outer_results.items():
    for tax_level_key, tax_level_value in data_type_value.items():
        for alg_key, alg_value in tax_level_value.items():
            for ind, value in enumerate(alg_value["inner_median_valid_AUROC"]):
                if abs(value - max_median_auroc) < tolerance:
                    best_models_info.append(
                        {
                            "data_type": data_type_key,
                            "tax_level": tax_level_key,
                            "alg": alg_key,
                            "params": alg_value["params"][ind],
                            "random_state": ind,
                            "outer_iter_no": ind,
                            "test_AUROC": alg_value["AUROC"][ind],
                        }
                    )

best_models_info

## Evaluate performance of the best model on the test set

In [ ]:
# We already calculated the performance of each model on the test set
# Access those scores for the best model
for num, model_info in enumerate(best_models_info, 1):
    print(f"#{num}: ")
    print(f"\tData Type: {model_info['data_type']}")
    print(f"\tTaxonomic Level: {model_info['tax_level']}")
    print(f"\tLearning Algorithm: {model_info['alg']}")
    print(f"\tAUROC: {model_info['test_AUROC']}")

## Evaluate performance of the best model on an external validation set

## Perform model interpretation

In [ ]:
# SHAP

## Perform error analysis

In [ ]:
# E.g.: Does the model perform better or worse on people of different ages, sex
# E.g.: Does the model have some sort of bias based on hospital center

## Ensure that the work is reproducible

In [ ]:
# Add talking points about how to make the work reproducible

## Conclusion

Provide example of what to report / language to use

## Works Cited

Andrews, S. "FastQC: a quality control tool for high throughput sequence data." (2010). Available online at: http://www.bioinformatics.babraham.ac.uk/projects/fastqc/.

Ewels, P., Magnusson, M., Lundin, S., Käller, M. "MultiQC: summarize analysis results for multiple tools and samples in a single report." *Bioinformatics* 32.19 (2016): 3047-3048.

Zhu, F., *et al*. "Metagenome-wide association of gut microbiome features for schizophrenia." *Nature communications* 11.1 (2020): 1612.

Langmead, B., Salzberg, S.L. "Fast gapped-read alignment with Bowtie 2." *Nature methods* 9.4 (2012): 357-359.